<a href="https://colab.research.google.com/github/450586509/Knowledge/blob/master/event_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings

warnings.filterwarnings('ignore')

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 16.0MB/s 
     |████████████████████████████████| 890kB 19.7MB/s 
     |████████████████████████████████| 3.0MB 24.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4f5d01846f79d84d78757c2a8c7df2fd909f1b597216d13508b7887eaa3acf57
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
dev_url = "http://129.204.205.246/event_entity_dev_data.csv"
train_url = "http://129.204.205.246/event_entity_train_data_label.csv"
train_df = pd.read_csv(train_url, delimiter='\t', header=None)
train_df.columns = ["id","text","event_name", "company_name"]

### 过滤长度大于500的行。

In [3]:
print(train_df.shape)
train_df["len"] = train_df.text.apply((lambda x: len(x)))
print(train_df.shape)
print(train_df.head)
for index, row in train_df.iterrows():
  print(f"""index={index} text={row.get("text")} len={len(row.get("text"))}""")
  if index > 5:
    break

(66761, 4)
(66761, 5)
<bound method NDFrame.head of             id  ...  len
0      2444634  ...   48
1      2836026  ...   60
2      2809128  ...   86
3      2221860  ...   62
4      2091205  ...   57
...        ...  ...  ...
66756  2568107  ...   63
66757  2688414  ...   53
66758  2415192  ...  349
66759  2983848  ...  322
66760  2834817  ...   98

[66761 rows x 5 columns]>
index=0 text=世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5%股份 len=48
index=1 text=LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45% 汇添富或为“失血门”跑路主力 len=60
index=2 text=四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再选 欧盟紧急声明要求希腊恪守承诺巨力索具(002342)下调预测 因产品毛利率下降 len=86
index=3 text=单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%好利来(002729)股东高位减持 len=62
index=4 text=深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反常”下滑 路演现场遭质疑 len=57
index=5 text=ST皇台(000995)总经理辞职销售公司总经理接任电连技术的“一年之痒”:高管辞职业绩变脸股价破发逾三成 len=53
index=6 text=国润新材因2017年报扔未披露被提示摘牌风险安德利头顶电商压力业绩下坡 自营模式存隐患中小板三公司发布高管辞职公告 len=57


In [4]:
train_df = train_df[train_df.len < 500]
print(f"""max_len={train_df["len"].max()}""")
print(f"""mean_len={train_df["len"].mean()}""")
print(f"shape={train_df.shape}")
train_df.head()

max_len=499
mean_len=86.30794621767306
shape=(66044, 5)


,id,text,event_name,company_name,len
0,2444634,世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5%股份,业绩下滑,世联君汇,48
1,2836026,LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45% ...,业绩下滑,华兰生物,60
2,2809128,四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再选...,业绩下滑,巨力索具,86
3,2221860,单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%好...,业绩下滑,北大荒,62
4,2091205,深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反常...,业绩下滑,龙大肉食,57


In [5]:
event_counts = train_df.event_name.value_counts()
print(f"type of event_counts = {type(event_counts)}")
event_counts

type of event_counts = <class 'pandas.core.series.Series'>


业务资产重组       7764
涉嫌非法集资       4195
股票转让-股权受让    3402
债务违约         3385
涉嫌传销         3107
实控人股东变更      2172
交易违规         2062
不能履职         1573
涉嫌欺诈         1504
涉嫌违法         1420
实际控制人变更      1402
重组失败         1283
业绩下滑          834
财务信息造假        719
提现困难          710
财务造假          705
资金紧张          587
商业信息泄露        550
实际控制人涉诉仲裁     519
歇业停业          458
失联跑路          384
高管负面          380
资产负面          375
资金账户风险        372
债务重组          342
投诉维权          296
履行连带担保责任       87
Name: event_name, dtype: int64

In [6]:
print(f"样本最少为{event_counts.min()}")
print(f"样本最多为{event_counts.max()}")
print(f"样本平均为{round(event_counts.mean(),4)}")

样本最少为87
样本最多为7764
样本平均为1503.2222


In [8]:
#model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-chinese')
model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [9]:
print(tokenizer.encode("hello world 谢 稳 文", add_special_tokens=True))
print(tokenizer.encode("hello world 谢 稳 文", add_special_tokens=False))
#tokenizer.encode("hello world你好啊", add_special_tokens=True)

[101, 7592, 2088, 100, 100, 1861, 102]
[7592, 2088, 100, 100, 1861]


In [10]:
train_tokenized = train_df.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(f"train_tokenized type = {type(train_tokenized)}")
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)


train_tokenized type = <class 'pandas.core.series.Series'>
train set shape: (66044, 491)


In [11]:
import torch.utils.data as Data
from sklearn import preprocessing

In [12]:
label = train_df.event_name.tolist()
le = preprocessing.LabelEncoder()
le.fit(label)
y = le.transform(label)
x = train_padded
print(type(y))
print(y[0:10])
print(label[0:10])
print(f"x shape ={x.shape}")
print(f"y shape ={y.shape}")

<class 'numpy.ndarray'>
[3 3 3 3 3 3 3 3 3 3]
['业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑', '业绩下滑']
x shape =(66044, 491)
y shape =(66044,)


In [22]:
import time
x = torch.tensor(x).long()
y = torch.tensor(y)
batch_size = 32
torch_dataset = Data.TensorDataset(x , y)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = batch_size,
    shuffle=False,
    num_workers = 2,  #采用两个进程来提取
)
train_x, train_y = None, None
cost = 0
model.to('cuda')
for step , (batch_x,batch_y) in enumerate(loader):
    # transform……
    start = time.time()
    
    bm = np.where(batch_x != 0, 1, 0)
    batch_mask = torch.tensor(bm).long()
    with torch.no_grad():
      batch_x = batch_x.to('cuda')
      batch_mask = batch_mask.to('cuda')
      train_last_hidden_states = model(batch_x, attention_mask=batch_mask)
      train_last_hidden_states = train_last_hidden_states[0].cpu()
    train_features = train_last_hidden_states[:,0,:].numpy()
    if train_x is None:
        train_x = train_features
        train_y = batch_y
    else:
        train_features = train_last_hidden_states[:,0,:].numpy()
        train_x = np.vstack((train_x, train_features))
        train_y = np.hstack((train_y,batch_y.numpy()))
    cur_cost = time.time() - start
    cost = cur_cost if step ==0 else (cost + cur_cost)/2
    if step % 50 == 0:
      print(f"step={step}")
      print(f"average cost = {cost}")
    

step=0
average cost = 2.0234692096710205
step=50
average cost = 1.8786270191770322
step=100
average cost = 1.8776800951313581
step=150
average cost = 1.8709770689800194
step=200
average cost = 1.878625819738628
step=250
average cost = 1.875736327313229
step=300
average cost = 1.877930058597045
step=350
average cost = 1.8706914369962895
step=400
average cost = 1.8889703843595589
step=450
average cost = 1.883848002504541
step=500
average cost = 1.8912273220286264
step=550
average cost = 1.8922359341817363
step=600
average cost = 1.893158453176192
step=650
average cost = 1.9033771937505903
step=700
average cost = 1.9012737303710234
step=750
average cost = 1.906532545104679
step=800
average cost = 1.91289394191976
step=850
average cost = 1.8985898054720038
step=900
average cost = 1.8953322008743165
step=950
average cost = 1.90434403867099
step=1000
average cost = 1.9029435565943622


KeyboardInterrupt: ignored

### 直接利用LogisticRegression模型训练。

In [ ]:
print(train_x.shape)
print(train_y.shape)
lr_clf = LogisticRegression()
lr_clf.fit(train_x, train_y)
lr_clf.score(train_x, train_y)

True